In [220]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import nltk

In [221]:
train=pd.read_csv("bases/working_train.csv",encoding="utf8")
test=pd.read_csv("bases/working_test.csv",encoding="utf8")

In [82]:
#LEAK PART FOR EVALUATING FEATURES :
###### OR USE THIS IF U WANT TO USE THE LEAKS!!!!! (not necessary!) ##############################s
pickle_in = open("Leaks/Leaks_all", "rb") #### OR use Leaks for the full leaks (no sub)
Leaks = pickle.load(pickle_in)

test_leak = Leaks
test_leak.index = test_leak.ID
Class_leaks = test_leak.Class
test =test[test.ID.isin(test_leak.index)] # drop Class, to be used for validation later on 

In [83]:
def evaluate_features(X_train,y_train,X_test,clf=None):
    if clf is None:
        clf = LogisticRegression()
    clf.fit(X_train,y_train)
    probas=clf.predict_proba(X_test)
    y = Class_leaks - 1
    print('Log loss: {}'.format(log_loss(y, probas)))

In [84]:
X_train=np.array(train.drop(["Class","ID","Text","Gene","Variation"],axis=1))
X_test=np.array(test.drop(["Text","Class","ID","Gene","Variation"],axis=1))
y_train=np.array(train["Class"])-1

In [85]:
train.columns

Index(['Class', 'Gene', 'ID', 'Text', 'Variation', 'Substitutions_var',
       'Stop_codon_var', 'Fusion_var', 'gene_fusion_var', 'Deletion_var',
       'del_or_ins_var', 'Amplification_var', 'Truncation_var', 'exon_var',
       'frameshift_var', 'dup_var', 'Gene_Share', 'Variation_Share',
       'Text_words', 'Figure_counter'],
      dtype='object')

In [86]:
#this function can be used to check if one variable could be useful. 
evaluate_features(X_train,y_train,X_test)

Log loss: 1.612383807108959


# Here we will add features from word2vec retrained then get the mean for the sentences

In [88]:
class MySentences(object):
    """MySentences is a generator to produce a list of tokenized sentences 
    
    Takes a list of numpy arrays containing documents.
    
    Args:
        arrays: List of arrays, where each element in the array contains a document.
    """
    def __init__(self, *arrays):
        self.arrays = arrays
 
    def __iter__(self):
        for array in self.arrays:
            for document in array:
                for sent in nltk.sent_tokenize(document):
                    yield nltk.word_tokenize(sent)

def get_word2vec(sentences, location):
    """Returns trained word2vec
    
    Args:
        sentences: iterator for sentences
        
        location (str): Path to save/load word2vec
    """
    if os.path.exists(location):
        print('Found {}'.format(location))
        model = gensim.models.Word2Vec.load(location)
        return model
    
    print('{} not found. training model'.format(location))
    model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)
    print('Model done training. Saving to disk')
    model.save(location)
    return model

In [263]:
count=0
for doc in train["Text"]:
    if type(doc)==float:
        print(count)
    count=count+1

1843


In [265]:
df_train["Text"][1843]

"The insulin-like growth factor I receptor (IGF1R) is overexpressed in several forms of human cancer, and it has emerged as an important target for anticancer drug design. Cancer genome sequencing efforts have recently identified three somatic mutations in IGF1R: A1374V, a deletion of S1278 in the C-terminal tail region of the receptor, and M1255I in the C-terminal lobe of the kinase catalytic domain. The possible effects of these mutations on IGF1R activity and biological function have not previously been tested. Here, we tested the effects of the mutations on the in vitro biochemical activity of IGF1R and on major IGF1R signaling pathways in mammalian cells. While the mutations do not affect the intrinsic tyrosine kinase activity of the receptor, we demonstrate that the basal (unstimulated) levels of MAP kinase and Akt activation are increased in the mutants (relative to wild-type IGF1R). We hypothesize that the enhanced signaling potential of these mutants is due to changes in prote

In [219]:
#It's important to remove duplicated spaces for word2vec learning !
train["Text"]=[" ".join(doc.split()) for doc in train["Text"].values]
test["Text"]=[" ".join(doc.split()) for doc in test["Text"].values]

AttributeError: 'float' object has no attribute 'split'

In [112]:
w2vec = get_word2vec(
    MySentences(
        train["Text"].values),"localisation"
)

localisation not found. training model


TypeError: expected string or bytes-like object

In [98]:
print(type(df_train["Text"].values))
print(type(train["Text"].values))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [111]:
type(train["Text"][10])

str

In [110]:
type(df_train["Text"][10])

str